In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Custom_Predict_and_Handler_SDK_Integration.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/prediction/custom_prediction_routines/SDK_Custom_Predict_and_Handler_SDK_Integration.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概览

本教程演示了如何使用Vertex AI SDK构建一个自定义容器，该容器使用Custom Prediction Routine模型服务器来为Vertex AI Predictions提供一个scikit-learn模型。



### 数据集

本教程使用了R.A. Fisher的鸢尾花数据集，这是一个小型数据集，常用于尝试机器学习技术。每个实例都有四个数值特征，这些特征是花的不同测量，并且有一个目标标签，将其标记为三种类型之一的鸢尾花：Iris setosa，Iris versicolour或Iris virginica。

本教程使用了[包含在scikit-learn库中的鸢尾花数据集的副本](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)。

### 目标

目标是：
- 训练一个模型，使用花的测量作为输入来预测它是什么类型的鸢尾花。
- 保存模型及其序列化的预处理器
- 使用Vertex AI SDK中的Custom Prediction Routine功能构建一个自定义的sklearn服务容器，包括自定义请求处理和预处理
- 在本地测试构建的容器
- 上传并部署自定义容器到Vertex Prediction

本教程更关注如何使用Vertex AI部署这个模型，而不是模型本身的设计。

### 成本

本教程使用了Google Cloud的计费组件：

* Vertex AI

了解有关[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)的信息，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预估使用量生成成本估算。

### 搭建本地开发环境

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。 您需要以下内容：

- Docker
- Git
- Google Cloud SDK (gcloud)
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud 的 [设置Python开发环境指南](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。 以下步骤提供了简明的指令：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。 激活虚拟环境。

4. 要安装Jupyter，请在终端窗口的命令行中运行 `pip install jupyter`。

5. 要启动Jupyter，请在终端窗口的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook Dashboard中打开此笔记本。

### 安装额外的包

安装额外的包依赖项，这些包可能在您的笔记本环境中尚未安装，比如NumPy、Scikit-learn、FastAPI、Uvicorn和joblib。

In [ ]:
%%writefile requirements.txt
fastapi
uvicorn==0.17.6
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0

您部署的模型预先安装了一组不同的依赖项，而不是您的笔记本环境已经安装的依赖项。您不应该假设因为笔记本中的工作正常，它们也会在模型中正常工作。相反，我们将通过在 requirements.txt 中列出它们，并使用 `pip install` 在笔记本中安装完全相同的依赖项来明确模型的依赖关系。请注意，当然，有可能我们错过了 requirements.txt 中已经存在于笔记本中的依赖项。如果是这种情况，这些功能将在笔记本中运行，但在模型中不会。为了防范这种情况，我们会在部署到云端之前在本地测试模型。

In [ ]:
# Install the same dependencies used in the serving container in the notebook
# environment.
%pip install -U --user -r requirements.txt

重新启动内核

在安装额外的包之后，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

在这个例子中设置日志记录

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用用于您的计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行此笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`或`%`为前缀的行视为shell命令，并将Python变量插入这些命令中的`$`或`{}`。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改 `REGION` 变量，该变量用于本笔记本的其余操作。以下是 Vertex AI 支持的区域。我们建议您选择距离您最近的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太: `asia-east1`

您可能不能使用多区域存储桶进行 Vertex AI 训练。不是所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

### 配置项目和资源名称

如果你正在参加一个现场教程会话，你可能会使用一个共享的测试账户或项目。为了避免资源创建时用户之间的名称冲突，你需要为每个实例会话创建一个时间戳，并将其附加到你在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

配置GCP资源名称。

In [ ]:
MODEL_ARTIFACT_DIR = "sklearn-cpr-model-handler-sdk"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction-handler-sdk"  # @param {type:"string"}
IMAGE = "sklearn-cpr-server-handler-sdk"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "sklearn-cpr-model-handler-sdk"  # @param {type:"string"}

`REGION` - 用于此笔记本其余操作。请确保选择一个[支持Cloud Vertex AI服务的地区](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)。您不能在Vertex AI中使用多区域存储桶进行预测。

`MODEL_ARTIFACT_DIR` - 您的模型工件所在的云存储桶文件夹路径，例如："my-models/fraud-detection/trial-4"

`REPOSITORY` - 要创建或使用的工件存储库的名称。

`IMAGE` - 将要推送的容器镜像的名称。

`MODEL_DISPLAY_NAME` - Vertex AI模型资源的显示名称。

### 创建一个云存储桶

**无论您使用的是哪个笔记本环境，都需要执行以下步骤。**

要更新模型工件而不重新构建容器，您必须将模型工件和任何自定义代码上传到云存储。

请在下方设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶中的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

编写您的预处理程序

决定放置所有自定义代码的目录。

In [ ]:
USER_SRC_DIR = "src_dir_handler_sdk"  # @param {type:"string"}

决定放置您训练模型的目录。

In [ ]:
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts"  # @param {type:"string"}

对训练数据进行缩放，使每个数值特征列的均值为0，标准差为1，可以改善您的模型。

创建`preprocess.py`，其中包含一个用于执行此缩放的类：

In [ ]:
%mkdir $USER_SRC_DIR
%mkdir $LOCAL_MODEL_ARTIFACTS_DIR

In [ ]:
%%writefile $USER_SRC_DIR/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds

## 使用预处理器训练和存储模型
接下来，使用`preprocess.MySimpleScaler`对鸢尾花数据进行预处理，然后使用scikit-learn训练模型。

最后，将训练好的模型导出为一个joblib（.joblib）文件，将您的`MySimpleScaler`实例导出为一个pickle（.pkl）文件。

In [ ]:
%cd $USER_SRC_DIR/

import pickle

import joblib
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(model, f"../{LOCAL_MODEL_ARTIFACTS_DIR}/model.joblib")
with open(f"../{LOCAL_MODEL_ARTIFACTS_DIR}/preprocessor.pkl", "wb") as f:
    pickle.dump(scaler, f)

### 将模型工件和自定义代码上传至云存储

在您部署模型进行服务之前，Vertex AI 需要访问以下云存储中的文件：

* `model.joblib`（模型工件）
* `preprocessor.pkl`（模型工件）

运行以下命令上传您的文件：

In [ ]:
%cd ..
!gsutil cp {LOCAL_MODEL_ARTIFACTS_DIR}/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
!gsutil ls {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

使用CPR模型服务器构建一个自定义的服务容器

现在模型和处理器已经训练并保存，是时候构建自定义的Serving容器了。通常构建一个Serving容器需要编写模型服务器代码。然而，使用自定义预测例程功能，Vertex AI Prediction提供一个可立即使用的模型服务器。

自定义Serving容器包含以下三部分代码：
1. [模型服务器](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/model_server.py)
    * 托管模型的HTTP服务器。
    * 负责设置路由/端口等。
1. [请求处理程序](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/handler.py)
    * 负责处理请求的Web服务器方面，例如对请求体进行反序列化、序列化响应、设置响应头等。
    * 在这个示例中，我们将使用SDK提供的默认处理程序 `google.cloud.aiplatform.prediction.handler.PredictionHandler`。
1. [预测器](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/predictor.py)
    * 负责处理预测请求的机器学习逻辑。

这三个部分可以根据自定义容器的需求进行定制。在这个示例中，我们将实现`Predictor`和`Handler`。

一个[`Predictor`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/predictor.py)必须实现以下接口：

```
from abc import ABC, abstractmethod
from typing import Any


class Predictor(ABC):
    """为自定义预测例程定义的 Predictor 类的接口。
    Predictor 负责处理预测请求的机器学习逻辑。
    具体地，Predictor 必须定义：
    (1) 如何将所有用于预测的模型工件加载到内存中。
    (2) 在预测时应执行的逻辑。
    当使用默认的 PredictionHandler 时，Predictor 将如下被调用：
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """

    def __init__(self):
        return

    @abstractmethod
    def load(self, artifacts_uri: str) -> None:
        """加载模型工件。
        Args:
            artifacts_uri (str):
                必需。环境变量 AIP_STORAGE_URI 的值。
        """
        pass

    def preprocess(self, prediction_input: Any) -> Any:
        """在进行预测之前预处理预测输入。
        Args:
            prediction_input (Any):
                必要。需预处理的预测输入。
        Returns:
            预处理后的预测输入。
        """
        return prediction_input

    @abstractmethod
    def predict(self, instances: Any) -> Any:
        """进行预测。
        Args:
            instances (Any):
                必要。用于进行预测的实例(们)。
        Returns:
            预测结果。
        """
        pass

    def postprocess(self, prediction_results: Any) -> Any:
        """后处理预测结果。
        Args:
            prediction_results (Any):
                必需。预测结果。
        Returns:
            后处理后的预测结果。
        """
        return prediction_results
```

首先，实现一个自定义的`Predictor`，它能够加载预处理器和模型。预处理器和模型将在`predict`时使用。

自定义预测例程支持一种在本地运行容器以测试图像的方法。在本地测试图像时，可以传递一个GCS路径或本地路径。
- 如果传递了一个GCS路径，则需要设置凭据。
- 如果想通过传递本地路径来进行测试，需要在`Predictor`中支持远程和本地加载模型。

Vertex SDK提供了一个名为`download_model_artifacts`的函数，可帮助您从GCS路径或本地路径下载模型工件。请参阅下面`load`函数中的示例。

In [ ]:
%%writefile $USER_SRC_DIR/predictor.py

import joblib
import numpy as np
import pickle

from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud.aiplatform.utils import prediction_utils

from sklearn.datasets import load_iris


class CprPredictor(Predictor):
    
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str):
        """Loads the preprocessor and model artifacts."""
        prediction_utils.download_model_artifacts(artifacts_uri)

        with open("preprocessor.pkl", "rb") as f:
            preprocessor = pickle.load(f)

        self._class_names = load_iris().target_names
        self._model = joblib.load("model.joblib")
        self._preprocessor = preprocessor

    def predict(self, instances):
        """Performs prediction."""
        inputs = np.asarray(instances)
        preprocessed_inputs = self._preprocessor.preprocess(inputs)
        outputs = self._model.predict(preprocessed_inputs)

        return {"predictions": [self._class_names[class_num] for class_num in outputs]}

然后实现自定义的`Handler`。

[`Handler`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/handler.py)必须实现以下接口。

```python
class Handler:
    """用于处理预测请求的 Handler 类的接口。"""

    def __init__(
        self, artifacts_uri: str, predictor: Optional[Type[Predictor]] = None,
    ):
        """初始化 Handler 实例。
        Args:
            artifacts_uri (str):
                必填。环境变量 AIP_STORAGE_URI 的值。
            predictor (Type[Predictor]):
                可选。如果给定的话，这个 Handler 使用的 Predictor 类来初始化预测器实例。
        """
        pass

    def handle(self, request: Request) -> Response:
        """处理预测请求。
        Args:
            request (Request):
                发送到应用程序的请求。
        Returns:
            预测请求的响应。
        """
        pass
```

然后实现一个自定义的`Handler`。

In [ ]:
%%writefile $USER_SRC_DIR/handler.py

import csv
from io import StringIO
import json

from fastapi import Response

from google.cloud.aiplatform.prediction.handler import PredictionHandler

class CprHandler(PredictionHandler):
    """Default prediction handler for the prediction requests sent to the application."""

    async def handle(self, request):
        """Handles a prediction request."""
        request_body = await request.body()
        prediction_instances = self._convert_csv_to_list(request_body)

        prediction_results = self._predictor.postprocess(
            self._predictor.predict(self._predictor.preprocess(prediction_instances))
        )

        return Response(content=json.dumps(prediction_results))
    
    def _convert_csv_to_list(self, data):
        """Converts list of string in csv format to list of float.
        
        Example input:
          b"1.1,2.2,3.3,4.4\n2.3,3.4,4.5,5.6\n"
          
        Example output:
            [
                [1.1, 2.2, 3.3, 4.4],
                [2.3, 3.4, 4.5, 5.6],
            ]
        """
        res = []
        for r in csv.reader(StringIO(data.decode("utf-8")), quoting=csv.QUOTE_NONNUMERIC):
            res.append(r)
        return res

构建自定义容器，我们还需要编写一个启动模型服务器的镜像入口点。然而，使用自定义预测例程功能，您不再需要编写入口点。Vertex SDK将使用您提供的自定义预测程序填充入口点。

将依赖项写入源目录，这些依赖项将安装在镜像中。

In [ ]:
!cp requirements.txt $USER_SRC_DIR/requirements.txt

构建并将容器推送到Artifact Registry。

### 构建您的自定义容器

要构建自定义图像，您需要一个Dockerfile，其中需要实现图像的外观。但是，使用自定义预测例程功能，Vertex SDK还会为您生成Dockerfile并构建图像。

默认情况下使用`python:3.7`作为基础图像。

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src_dir_handler_sdk.handler import \
    CprHandler  # Update this path as the variable $USER_SRC_DIR to import the custom handler.
from src_dir_handler_sdk.predictor import \
    CprPredictor  # Update this path as the variable $USER_SRC_DIR to import the custom predictor.

local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CprPredictor,  # Update this to the custom predictor class.
    handler=CprHandler,  # Update this to the custom handler class.
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
)

你可以查看构建图像的服务容器规格。

In [ ]:
local_model.get_serving_container_spec()

### 存储测试实例

要发送 CSV 格式的输入实例，需要使用原始预测来使用任意的 HTTP 载荷，而不是 JSON 格式。[阅读文档。](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.endpoints/rawPredict)

In [ ]:
INPUT_FILE = "instances.csv"

In [ ]:
%%writefile $INPUT_FILE
6.7,3.1,4.7,1.5
4.6,3.1,1.5,0.2

### 本地运行和测试容器

自定义预测例程提供两种下载模型工件以便在本地测试图像的方式。
1. 本地路径。
2. GCS路径。

要使用本地路径，您的“预测器”需要支持两种加载模型的方式，以便可以在本地进行测试并部署到Vertex AI预测服务。SDK提供了一个名为`download_model_artifacts`的方法，以支持[prediction_utils.py](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py)中的这两种方式，您可以在“预测器”的`load`函数中调用它。

如果您想使用GCS路径在本地测试图像，您需要按照说明设置凭据。

设置凭证

设置凭证仅在本地运行自定义服务容器时需要使用GCS路径。设置凭证是为了执行`Predictor`的`load`函数，该函数会从Google Cloud Storage下载模型文物。

要在您的项目中访问Google Cloud Storage，您需要通过以下方式之一设置凭证：

1. 用户账户

2. 服务账户

您可以在[这里](https://cloud.google.com/docs/authentication#principals)了解更多关于以上各项的信息。

选项1. 使用Google用户凭据

首先授权Google授权库访问具有Google用户凭据的云平台。此步骤涉及一个交互式提示，需要用户输入。如果您正在使用非交互式shell，您应该打开一个终端来运行此命令。请参阅[此处](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html)了解如何在Vertex Workbench笔记本环境中打开终端。

请注意，如果您在没有窗口管理器/浏览器的计算机上运行下面的命令，它将提示您运行命令`gcloud auth application-default login --remote-bootstrap="..."`。只有可以启动Web浏览器的计算机，例如笔记本电脑或工作站，被允许运行此命令。完成使用浏览器的身份验证步骤后，您需要将验证代码复制并粘贴回原始终端以继续登录流程。如果您在具有Web浏览器支持的计算机上运行此命令，则无需使用另一台机器。

此命令将打印凭据文件为`Credentials saved to file: [CREDENTIALS_FILE]`。

In [ ]:
!gcloud auth application-default login

指定凭据文件，该文件为json文件，作为路径。

In [ ]:
CREDENTIALS_FILE = "[CREDENTIALS_FILE]"  # @param {type:"string"}

接下来，授权gcloud使用Google用户凭据访问Cloud平台。用户凭据需要在项目中具有`setIamPolicy`权限。更多详细信息请参见[这里](https://cloud.google.com/resource-manager/docs/access-control-proj)。

与上一步类似，这也需要用户输入。请按照前一步的说明继续操作。

In [ ]:
!gcloud auth login

为用户帐户授予角色。使用您在 gcloud 认证步骤中使用的电子邮件地址，例如 `myemail@gmail.com`。由于我们将使用此用户帐户从 Google Cloud 存储下载模型工件，因此我们授予用户帐户** Storage Object Viewer** 角色。有关更多详细信息，请参阅[Cloud Storage 的 IAM 角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
USER_ACCOUNT = "[USER_ACCOUNT]"  # @param {type:"string"}

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=user:$USER_ACCOUNT --role=roles/storage.objectViewer

使用项目ID初始化Vertex AI SDK，项目ID是使用用户凭据所必需的。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

选项2. 使用谷歌服务账号凭据

如果您已经完成了上面的选项1，请跳过此步骤。如果尚未启用IAM API，请先启用它。

In [ ]:
!gcloud services enable iam.googleapis.com

指定在您的项目中要唯一的服务账户名称。

In [ ]:
SERVICE_ACCOUNT = "[SERVICE_ACCOUNT]"  # @param {type:"string"}

创建服务账号。

In [ ]:
!gcloud iam service-accounts create $SERVICE_ACCOUNT

授权gcloud使用Google用户凭据访问云平台。用户凭据需要在项目中具有`setIamPolicy`权限。有关更多详细信息，请参阅[这里](https://cloud.google.com/resource-manager/docs/access-control-proj)。

这一步涉及一个交互式提示，需要用户输入。如果您使用的是非交互式shell，您应该打开一个终端来运行此命令。请参见[这里](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html) 如何在Vertex Workbench笔记本环境中打开终端。

请注意，如果您在没有窗口管理器/浏览器的机器上运行下面的命令，它将提示您运行命令 `gcloud auth application-default login --remote-bootstrap="..."`。只有能够启动Web浏览器的机器，例如笔记本电脑或工作站，可以运行此命令。完成浏览器上的身份验证步骤后，您需要将验证代码复制并粘贴回原始终端以继续登录流程。如果您在支持Web浏览器的机器上运行此命令，则无需使用一个单独的机器。

In [ ]:
!gcloud auth login

为服务账号授权角色。由于我们将使用此服务账号从Google Cloud Storage下载模型工件，我们授予服务账号“Storage Object Viewer”角色。详细信息请参阅[Cloud Storage的IAM角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com \
    --role=roles/storage.objectViewer

指定服务帐号密钥文件名。

In [ ]:
CREDENTIALS_FILE = "./credentials.json"

生成服务账号密钥文件。在本地运行自定义服务容器时会用到。

In [ ]:
!gcloud iam service-accounts keys create $CREDENTIALS_FILE \
    --iam-account="{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com"

#### 在本地运行并发送请求到容器

通过自定义预测例程功能，可以很容易地在本地测试容器。

在这个例子中，容器执行一个预测请求和一个健康检查。

选项1. 通过本地路径传递

这个选项要求`Predictor`中的`load`函数支持从GCS路径和本地路径加载模型文件。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应及其内容。

In [ ]:
predict_response, predict_response.content

打印出健康检查的响应和内容。

In [ ]:
health_check_response, health_check_response.content

也请打印出所有的容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

选项2. 传递GCS路径

如果您想传递GCS路径，则需要在先前步骤中设置凭据，并在运行容器时传递凭据的路径。服务账号应该具有**存储对象查看器**权限。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应及其内容。

In [ ]:
predict_response, predict_response.content

打印出健康检查的响应和其内容。

In [ ]:
health_check_response, health_check_response.content

也打印出所有容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### 将容器推送到构件存储库

配置Docker以访问构件存储库。然后将您的容器镜像推送到您的构件存储库仓库中。

In [ ]:
!gcloud services list

如果在您的项目中未启用`artifactregistry.googleapis.com`，请在继续之前启用该API。

In [ ]:
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

使用SDK推送图像。

In [ ]:
local_model.push_image()

部署到 Vertex AI

### 上传自定义容器模型

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

使用LocalModel实例来上传模型。它会自动为您填充容器规范。

In [ ]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
)

### 在Vertex AI上部署模型
完成这一步后，模型将被部署并准备好进行在线预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送预测

需要使用[原始预测](https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/raw-predict)，因为请求是以CSV格式而不是JSON格式。

### 使用Python SDK

In [ ]:
ENDPOINT_RESOURCE_NAME = endpoint.resource_name

In [ ]:
from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

prediction_client = aiplatform_v1.PredictionServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

with open(INPUT_FILE) as f:
    http_body = httpbody_pb2.HttpBody(
        data=f.read().encode("utf-8"),
        content_type="text/csv",
    )

request = aiplatform_v1.RawPredictRequest(
    endpoint=ENDPOINT_RESOURCE_NAME,
    http_body=http_body,
)

prediction_client.raw_predict(request=request)

使用REST

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: text/csv" \
--data-binary @instances.csv \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:rawPredict

### 使用gcloud CLI

In [ ]:
!gcloud ai endpoints raw-predict $ENDPOINT_ID \
  --region=$REGION \
  --http-headers=Content-Type=text/csv \
  --request=@$INPUT_FILE

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI